# SCENIC+

# Database

All the steps were done in command lne, but I copy them here for posterity

In [ ]:
git clone https://github.com/aertslab/create_cisTarget_databases

## Cluster buster download

In [ ]:
wget https://resources.aertslab.org/cistarget/programs/cbust
chmod a+x cbust

## Motif collection download

In [ ]:
mkdir -p aertslab_motif_colleciton
wget -O aertslab_motif_colleciton/v10nr_clust_public.zip https://resources.aertslab.org/cistarget/motif_collections/v10nr_clust_public/v10nr_clust_public.zip

In [ ]:
cd aertslab_motif_colleciton; unzip -q v10nr_clust_public.zip
cd ..

In [1]:
!ls aertslab_motif_colleciton/v10nr_clust_public/snapshots/


motifs-v10-nr.chicken-m0.00001-o0.0.tbl  motifs-v10-nr.hgnc-m0.00001-o0.0.tbl
motifs-v10-nr.flybase-m0.00001-o0.0.tbl  motifs-v10-nr.mgi-m0.00001-o0.0.tbl


In [3]:
!ls -l aertslab_motif_colleciton/v10nr_clust_public/singletons | head

total 42412
-rw-r--r-- 1 vsc33857 nogroup   163 Jan 27  2022 bergman__Adf1.cb
-rw-r--r-- 1 vsc33857 nogroup    75 Jan 27  2022 bergman__Aef1.cb
-rw-r--r-- 1 vsc33857 nogroup    75 Jan 27  2022 bergman__Hr46.cb
-rw-r--r-- 1 vsc33857 nogroup   113 Jan 27  2022 bergman__Kr.cb
-rw-r--r-- 1 vsc33857 nogroup    86 Jan 27  2022 bergman__Su_H_.cb
-rw-r--r-- 1 vsc33857 nogroup    83 Jan 27  2022 bergman__TFAM.cb
-rw-r--r-- 1 vsc33857 nogroup    77 Jan 27  2022 bergman__ap.cb
-rw-r--r-- 1 vsc33857 nogroup   398 Jan 27  2022 bergman__bcd.cb
-rw-r--r-- 1 vsc33857 nogroup    84 Jan 27  2022 bergman__bin.cb
ls: write error: Broken pipe


## Fasta from consensus regions

In [ ]:
module load BEDTools/2.31.1

REGION_BED="../atac/outs/consensus_peak_calling/consensus_regions.bed"
GENOME_FASTA="/genomes/homo_sapiens/hg38_ucsc/fasta/hg38.fa"
CHROMSIZES="/genomes/homo_sapiens/hg38_ucsc/fasta/hg38.chrom.sizes"
DATABASE_PREFIX="10x_ONT_BALL"
SCRIPT_DIR="/grn/create_cisTarget_databases"

${SCRIPT_DIR}/create_fasta_with_padded_bg_from_bed.sh \
        ${GENOME_FASTA} \
        ${CHROMSIZES} \
        ${REGION_BED} \
        hg38.BALL.with_1kb_bg_padding.fa \
        1000 \
        yes
        

## Create cistarget database

In [3]:
ls aertslab_motif_colleciton/v10nr_clust_public/singletons > motifs.txt

In [ ]:
OUT_DIR="/grn/"
CBDIR="${OUT_DIR}/aertslab_motif_colleciton/v10nr_clust_public/singletons"
FASTA_FILE="${OUT_DIR}/hg38.BALL.with_1kb_bg_padding.fa"
MOTIF_LIST="${OUT_DIR}/motifs.txt"
DATABASE_PREFIX="10x_ONT_BALL"

"${OUT_DIR}/create_cisTarget_databases/create_cistarget_motif_databases.py" \
    -f ${FASTA_FILE} \
    -M ${CBDIR} \
    -m ${MOTIF_LIST} \
    -o ${OUT_DIR}/${DATABASE_PREFIX} \
    --bgpadding 1000 \
    -t 20

# RNA preprocessing

Save rna counts in adata.raw and change RNA index barcode to ATAC one

In [4]:
import scanpy as sc

In [ ]:
rna = sc.read_h5ad("/path/to/rna/concatenated_adata_log_pca_celltypist_23012025.h5ad")

In [6]:
rna.layers

Layers with keys: counts, log_counts

In [7]:
rna.obs

,sample,n_genes_by_counts,log1p_n_genes_by_counts,total_counts,log1p_total_counts,pct_counts_in_top_20_genes,pct_counts_mt,pct_counts_ribo,pct_counts_hb,outlier,...,leiden_1.25,leiden_1.5,leiden_2.0,leiden_5.0,leiden_10.0,leiden_20.0,celltypist_cell_label_coarse,celltypist_conf_score_coarse,celltypist_cell_label_fine,celltypist_conf_score_fine
AAACAGCCAGTTTGTG-D0,D0,1891,7.545390,7641.0,8.941415,21.823060,13.584609,7.642979,0.0,False,...,3,4,2,7,39,294,B-cell lineage,0.978114,Pro-B cells,0.950723
AAACAGCCATGTTTGG-D0,D0,2319,7.749322,10986.0,9.304468,21.354451,13.899508,9.111597,0.0,False,...,7,7,8,6,73,51,B-cell lineage,0.603815,Pro-B cells,0.370262
AAACAGCCATTTAAGC-D0,D0,1667,7.419381,6636.0,8.800415,21.503918,13.803495,6.268836,0.0,False,...,3,4,2,2,24,29,B-cell lineage,0.951335,Pro-B cells,0.903238
AAACATGCATTGGGAG-D0,D0,1599,7.377759,6888.0,8.837681,24.811266,18.626598,5.894309,0.0,False,...,3,4,2,7,31,69,B-cell lineage,0.121671,Pro-B cells,0.183371
AAACCAACAATAACCT-D0,D0,1403,7.247081,5495.0,8.611775,24.913558,18.817106,5.150136,0.0,False,...,3,4,2,2,4,9,B-cell lineage,0.991872,Pro-B cells,0.996793
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTGTGAAGCGAGTAA-Q3,Q3,1361,7.216709,5059.0,8.529121,23.739870,17.019173,8.381103,0.0,False,...,1,1,11,30,16,54,B-cell lineage,0.403587,Pro-B cells,0.973642
TTTGTGAAGCTTTGGG-Q3,Q3,2369,7.770645,8738.5,9.075608,15.860846,9.990273,4.691881,0.0,False,...,1,1,0,0,58,91,B cells,0.776546,Pro-B cells,0.532727
TTTGTGGCAGCCTGCA-Q3,Q3,1536,7.337588,6272.0,8.744010,29.432398,23.852041,6.505102,0.0,False,...,1,1,0,0,58,91,B cells,0.484664,Pro-B cells,0.177973
TTTGTTGGTAAACAAG-Q3,Q3,1279,7.154615,5301.0,8.575839,35.653650,28.730429,2.886248,0.0,False,...,11,11,12,23,57,20,Monocytes,0.745200,Non-classical monocytes,0.041574


In [ ]:
# import multiome barcodes
import pandas as pd
atac_CB = pd.read_csv('/res_00001/barcodes/cellranger_arc_atac.737K-arc-v1.txt.gz', header = None)
rna_CB = pd.read_csv('/res_00001/barcodes/cellranger_arc_rna.737K-arc-v1.txt.gz', header = None)
rna_CB.rename(columns={0: 'rna_CB'}, inplace=True)
atac_CB.rename(columns={0: 'atac_CB'}, inplace=True)
df_merged =  pd.concat([rna_CB, atac_CB], axis=1)

In [10]:
rna.obs['base_index'] = rna.obs.index.str.split('-').str[0]
mapping_dict = pd.Series(df_merged['atac_CB'].values, index=df_merged['rna_CB']).to_dict()
rna.obs['atac_CB'] = rna.obs['base_index'].map(mapping_dict)
rna.obs['sample'] = rna.obs['sample'].str.split('_').str[0]
rna.obs['atac_CB'] = rna.obs['atac_CB'].astype(str)+'-'+rna.obs['sample'].astype(str) + '___' + rna.obs['sample'].astype(str)
rna.obs.set_index('atac_CB', inplace=True)
rna.obs

,sample,n_genes_by_counts,log1p_n_genes_by_counts,total_counts,log1p_total_counts,pct_counts_in_top_20_genes,pct_counts_mt,pct_counts_ribo,pct_counts_hb,outlier,...,leiden_1.5,leiden_2.0,leiden_5.0,leiden_10.0,leiden_20.0,celltypist_cell_label_coarse,celltypist_conf_score_coarse,celltypist_cell_label_fine,celltypist_conf_score_fine,base_index
atac_CB,,,,,,,,,,,,,,,,,,,,,
ACAGCGGGTGTTACTA-D0___D0,D0,1891,7.545390,7641.0,8.941415,21.823060,13.584609,7.642979,0.0,False,...,4,2,7,39,294,B-cell lineage,0.978114,Pro-B cells,0.950723,AAACAGCCAGTTTGTG
ACAGCGGGTTTATCCT-D0___D0,D0,2319,7.749322,10986.0,9.304468,21.354451,13.899508,9.111597,0.0,False,...,7,8,6,73,51,B-cell lineage,0.603815,Pro-B cells,0.370262,AAACAGCCATGTTTGG
ACAGCGGGTTCACTTT-D0___D0,D0,1667,7.419381,6636.0,8.800415,21.503918,13.803495,6.268836,0.0,False,...,4,2,2,24,29,B-cell lineage,0.951335,Pro-B cells,0.903238,AAACAGCCATTTAAGC
CATTTAGGTGTCCACA-D0___D0,D0,1599,7.377759,6888.0,8.837681,24.811266,18.626598,5.894309,0.0,False,...,4,2,7,31,69,B-cell lineage,0.121671,Pro-B cells,0.183371,AAACATGCATTGGGAG
CTTTATCGTCGAGGCA-D0___D0,D0,1403,7.247081,5495.0,8.611775,24.913558,18.817106,5.150136,0.0,False,...,4,2,2,4,9,B-cell lineage,0.991872,Pro-B cells,0.996793,AAACCAACAATAACCT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GTAGGTGTCGGTCCAT-Q3___Q3,Q3,1361,7.216709,5059.0,8.529121,23.739870,17.019173,8.381103,0.0,False,...,1,11,30,16,54,B-cell lineage,0.403587,Pro-B cells,0.973642,TTTGTGAAGCGAGTAA
GTAGGTGTCCCAATAT-Q3___Q3,Q3,2369,7.770645,8738.5,9.075608,15.860846,9.990273,4.691881,0.0,False,...,1,0,0,58,91,B cells,0.776546,Pro-B cells,0.532727,TTTGTGAAGCTTTGGG
GTTTGGTGTTTCTAAG-Q3___Q3,Q3,1536,7.337588,6272.0,8.744010,29.432398,23.852041,6.505102,0.0,False,...,1,0,0,58,91,B cells,0.484664,Pro-B cells,0.177973,TTTGTGGCAGCCTGCA


In [11]:
rna.X = rna.layers['counts'].copy()
rna.raw = rna

In [ ]:
rna.write_h5ad("/grn/concatenated_adata_log_pca_celltypist_23012025_with_raw_counts.BALL.h5ad")

# run scenicplus

In [13]:
! scenicplus


   ____   ____ _____ _   _ ___ ____      
  / ___| / ___| ____| \ | |_ _/ ___| _ 
  \___ \| |   |  _| |  \| || | |   _|.|_
   ___) | |___| |___| |\  || | |__|_..._|
  |____/ \____|_____|_| \_|___\____||_| 


scenicplus verions: 1.0a1
usage: scenicplus [-h] {init_snakemake,prepare_data,grn_inference} ...

Single-Cell Enhancer-driven gene regulatory Network Inference and Clustering

positional arguments:
  {init_snakemake,prepare_data,grn_inference}

options:
  -h, --help            show this help message and exit


In [14]:
!scenicplus init_snakemake --out_dir scplus_pipeline

2025-04-10 13:46:01,688 SCENIC+      INFO     Creating snakemake folder in: scplus_pipeline


In [15]:
!tree scplus_pipeline/

/bin/bash: line 1: tree: command not found


```
tree -d scplus_pipeline/
scplus_pipeline/
└── Snakemake
    ├── config
    └── workflow
```

In [16]:
!mkdir -p outs
!mkdir -p tmp

In [ ]:
! ls

In [18]:
!cd scplus_pipeline/Snakemake/

In [21]:
!cat scplus_pipeline/Snakemake/config/config.yaml

input_data:
  cisTopic_obj_fname: "/staging/leuven/stg_00096/home/projects/2023_Cools_B-ALL/atac/outs/cistopic_obj.pkl"
  GEX_anndata_fname: "/staging/leuven/stg_00096/home/projects/2023_Cools_B-ALL/scenic_plus/concatenated_adata_log_pca_celltypist_23012025_with_raw_counts.BALL.h5ad"
  region_set_folder: "/staging/leuven/stg_00096/home/projects/2023_Cools_B-ALL/atac/outs/region_sets"
  ctx_db_fname: "/staging/leuven/stg_00096/home/projects/2023_Cools_B-ALL/scenic_plus/10x_ONT_BALL.regions_vs_motifs.rankings.feather"
  dem_db_fname: "/staging/leuven/stg_00096/home/projects/2023_Cools_B-ALL/scenic_plus/10x_ONT_BALL.regions_vs_motifs.scores.feather"
  path_to_motif_annotations: "/staging/leuven/stg_00096/home/projects/2023_Cools_B-ALL/scenic_plus/aertslab_motif_colleciton/v10nr_clust_public/snapshots/motifs-v10-nr.hgnc-m0.00001-o0.0.tbl"

output_data:
  # output for prepare_GEX_ACC .h5mu
  combined_GEX_ACC_mudata: "/staging/leuven/stg_00096/home/projects/2023_Cools_B-ALL/scenic_plus/outs/

In [ ]:
!pwd

In [ ]:
cd /scplus_pipeline/Snakemake/

In [25]:
! snakemake --cores 24

Assuming unrestricted shared filesystem usage for local execution.
Building DAG of jobs...
Using shell: /usr/bin/bash
Provided cores: 24
Rules claiming more threads will be scaled down.
Job stats:
job                            count
---------------------------  -------
AUCell_direct                      1
AUCell_extended                    1
all                                1
download_genome_annotations        1
eGRN_direct                        1
eGRN_extended                      1
get_search_space                   1
motif_enrichment_cistarget         1
motif_enrichment_dem               1
prepare_GEX_ACC_multiome           1
prepare_menr                       1
region_to_gene                     1
scplus_mudata                      1
tf_to_gene                         1
total                             14

Select jobs to execute...
Execute 3 jobs...

[Thu Apr 10 13:53:12 2025]
localrule motif_enrichment_cistarget:
    input: /staging/leuven/stg_00096/home/projects/2023_Cools_B

In [26]:
! cat .snakemake/log/2025-04-10T135312.126428.snakemake.log

Building DAG of jobs...
Using shell: /usr/bin/bash
Provided cores: 24
Rules claiming more threads will be scaled down.
Job stats:
job                            count
---------------------------  -------
AUCell_direct                      1
AUCell_extended                    1
all                                1
download_genome_annotations        1
eGRN_direct                        1
eGRN_extended                      1
get_search_space                   1
motif_enrichment_cistarget         1
motif_enrichment_dem               1
prepare_GEX_ACC_multiome           1
prepare_menr                       1
region_to_gene                     1
scplus_mudata                      1
tf_to_gene                         1
total                             14

Select jobs to execute...
Execute 3 jobs...

[Thu Apr 10 13:53:12 2025]
localrule motif_enrichment_cistarget:
    input: /staging/leuven/stg_00096/home/projects/2023_Cools_B-ALL/atac/outs/region_sets, /staging/leuven/stg_00096/home/projects

In [27]:
pip list

Package                              Version
------------------------------------ -----------------
adjustText                           1.0.4
aiohttp                              3.9.3
aiosignal                            1.3.1
anndata                              0.10.5.post1
annoy                                1.17.3
appdirs                              1.4.4
arboreto                             0.1.6
argparse-dataclass                   2.0.0
array_api_compat                     1.5.1
asttokens                            2.4.1
attr                                 0.3.2
attrs                                23.2.0
bbknn                                1.6.0
beautifulsoup4                       4.12.3
bidict                               0.23.1
bioservices                          1.11.2
blosc2                               2.5.1
bokeh                                3.4.0
boltons                              23.1.1
bs4                                  0.0.2
cattrs                     